# Clasificación de Piso en el Dataset UJIIndoorLoc

---

## Introducción

En este notebook se implementa un flujo completo de procesamiento y análisis para la clasificación del **piso** en un entorno interior utilizando el dataset **UJIIndoorLoc**. Este conjunto de datos contiene mediciones de señales WiFi recopiladas en distintas ubicaciones de un edificio, con información sobre coordenadas, piso, usuario, hora, entre otros.

En esta tarea nos enfocaremos en predecir el **piso** en el que se encuentra un dispositivo, considerando únicamente las muestras etiquetadas con valores válidos para dicha variable. Se tratará como un problema de clasificación multiclase (planta baja, primer piso, segundo piso).

## Objetivos

- **Cargar y explorar** el conjunto de datos UJIIndoorLoc.
- **Preparar** los datos seleccionando las características relevantes y el target (`FLOOR`).
- **Dividir** el dataset en entrenamiento y validación (80/20).
- **Entrenar y optimizar** clasificadores basados en seis algoritmos:
  - K-Nearest Neighbors (KNN)
  - Gaussian Naive Bayes
  - Regresión Logística
  - Árboles de Decisión
  - Support Vector Machines (SVM)
  - Random Forest
- **Seleccionar hiperparámetros óptimos** para cada modelo utilizando validación cruzada (5-fold), empleando estrategias como **Grid Search**, **Randomized Search**, o **Bayesian Optimization** según el algoritmo.
- **Comparar el desempeño** de los modelos sobre el conjunto de validación, usando métricas como *accuracy*, *precision*, *recall*, y *F1-score*.
- **Determinar el mejor clasificador** para esta tarea, junto con sus hiperparámetros óptimos.

Este ejercicio permite no solo evaluar la capacidad predictiva de distintos algoritmos clásicos de clasificación, sino también desarrollar buenas prácticas en validación de modelos y selección de hiperparámetros en contextos del mundo real.

---

## Descripción del Dataset

El dataset utilizado en este análisis es el **UJIIndoorLoc Dataset**, ampliamente utilizado para tareas de localización en interiores a partir de señales WiFi. Está disponible públicamente en la UCI Machine Learning Repository y ha sido recopilado en un entorno real de un edificio universitario.

Cada muestra corresponde a una observación realizada por un dispositivo móvil, donde se registran las intensidades de señal (RSSI) de más de 500 puntos de acceso WiFi disponibles en el entorno. Además, cada fila contiene información contextual como la ubicación real del dispositivo (coordenadas X e Y), el piso, el edificio, el identificador del usuario, y la marca temporal.

El objetivo en esta tarea es predecir el **piso** (`FLOOR`) en el que se encontraba el dispositivo en el momento de la medición, considerando únicamente las características numéricas provenientes de las señales WiFi.

### Estructura del dataset

- **Número de muestras**: ~20,000
- **Número de características**: 520
  - 520 columnas con valores de intensidad de señal WiFi (`WAP001` a `WAP520`)
- **Variable objetivo**: `FLOOR` (variable categórica con múltiples clases, usualmente entre 0 y 4)

### Columnas relevantes

- `WAP001`, `WAP002`, ..., `WAP520`: niveles de señal recibida desde cada punto de acceso WiFi (valores entre -104 y 0, o 100 si no se detectó).
- `FLOOR`: clase objetivo a predecir (nivel del edificio).
- (Otras columnas como `BUILDINGID`, `SPACEID`, `USERID`, `TIMESTAMP`, etc., pueden ser ignoradas o utilizadas en análisis complementarios).

### Contexto del problema

La localización en interiores es un problema complejo en el que tecnologías como el GPS no funcionan adecuadamente. Los sistemas basados en WiFi han demostrado ser una alternativa efectiva para estimar la ubicación de usuarios en edificios. Poder predecir automáticamente el piso en el que se encuentra una persona puede mejorar aplicaciones de navegación en interiores, accesibilidad, gestión de emergencias y servicios personalizados. Este tipo de problemas es típicamente abordado mediante algoritmos de clasificación multiclase.


### Estrategia de evaluación

En este análisis seguiremos una metodología rigurosa para garantizar la validez de los resultados:

1. **Dataset de entrenamiento**: Se utilizará exclusivamente para el desarrollo, entrenamiento y optimización de hiperparámetros de todos los modelos. Este conjunto será dividido internamente en subconjuntos de entrenamiento y validación (80/20) para la selección de hiperparámetros mediante validación cruzada.

2. **Dataset de prueba**: Se reservará únicamente para la **evaluación final** de los modelos ya optimizados. Este conjunto **no debe ser utilizado** durante el proceso de selección de hiperparámetros, ajuste de modelos o toma de decisiones sobre la arquitectura, ya que esto introduciría sesgo y comprometería la capacidad de generalización estimada.

3. **Validación cruzada**: Para la optimización de hiperparámetros se empleará validación cruzada 5-fold sobre el conjunto de entrenamiento, lo que permitirá una estimación robusta del rendimiento sin contaminar los datos de prueba.

Esta separación estricta entre datos de desarrollo y evaluación final es fundamental para obtener una estimación realista del rendimiento que los modelos tendrían en un escenario de producción con datos completamente nuevos.

---


## Paso 1: Cargar y explorar el dataset

**Instrucciones:**
- Descarga el dataset **UJIIndoorLoc** desde la UCI Machine Learning Repository o utiliza la versión proporcionada en el repositorio del curso (por ejemplo: `datasets\UJIIndoorLoc\trainingData.csv`).
- Carga el dataset utilizando `pandas`.
- Muestra las primeras filas del dataset utilizando `df.head()`.
- Imprime el número total de muestras (filas) y características (columnas).
- Verifica cuántas clases distintas hay en la variable objetivo `FLOOR` y cuántas muestras tiene cada clase (`df['FLOOR'].value_counts()`).


In [1]:
import pandas as pd
from pathlib import Path

# Paso 1: Cargar y explorar el dataset UJIIndoorLoc (trainingData.csv)

# Posibles rutas comunes donde puede estar el archivo
candidate_paths = [
    Path("datasets/UJIIndoorLoc/trainingData.csv"),
    Path("datasets/UJIIndoorLoc/trainingData.csv").resolve(),
    Path("trainingData.csv"),
    Path("data/trainingData.csv"),
    Path("./trainingData.csv"),
]

df = None
loaded_path = None
for p in candidate_paths:
    if p.exists():
        try:
            df = pd.read_csv(p)
            loaded_path = p
            break
        except Exception as e:
            # si existe pero no se pudo leer, mostrar error y seguir probando otras rutas
            print(f"Error leyendo {p}: {e}")

if df is None:
    raise FileNotFoundError(
        "No se encontró trainingData.csv en las rutas probadas. "
        "Coloca el archivo en el directorio del notebook o en 'datasets/UJIIndoorLoc/'."
    )

# Mostrar resultados básicos de exploración
print(f"Archivo cargado: {loaded_path}")
print("\nPrimeras 5 filas:")
display(df.head())

print("\nDimensiones (filas, columnas):", df.shape)

print("\nColumnas (primeras 20 mostradas):")
print(list(df.columns)[:20], ("... total columnas=" + str(len(df.columns))))

if 'FLOOR' in df.columns:
    print("\nConteo por clase en 'FLOOR':")
    print(df['FLOOR'].value_counts(dropna=False))
    print("\nNúmero de clases distintas en 'FLOOR':", df['FLOOR'].nunique())
else:
    print("\nLa columna 'FLOOR' no se encontró en el dataset. Revisa el archivo.")

Archivo cargado: trainingData.csv

Primeras 5 filas:


,WAP001,WAP002,WAP003,WAP004,WAP005,WAP006,WAP007,WAP008,WAP009,WAP010,...,WAP520,LONGITUDE,LATITUDE,FLOOR,BUILDINGID,SPACEID,RELATIVEPOSITION,USERID,PHONEID,TIMESTAMP
0,100,100,100,100,100,100,100,100,100,100,...,100,-7541.2643,4.864921e+06,2,1,106,2,2,23,1371713733
1,100,100,100,100,100,100,100,100,100,100,...,100,-7536.6212,4.864934e+06,2,1,106,2,2,23,1371713691
2,100,100,100,100,100,100,100,-97,100,100,...,100,-7519.1524,4.864950e+06,2,1,103,2,2,23,1371714095
3,100,100,100,100,100,100,100,100,100,100,...,100,-7524.5704,4.864934e+06,2,1,102,2,2,23,1371713807
4,100,100,100,100,100,100,100,100,100,100,...,100,-7632.1436,4.864982e+06,0,0,122,2,11,13,1369909710



Dimensiones (filas, columnas): (19937, 529)

Columnas (primeras 20 mostradas):
['WAP001', 'WAP002', 'WAP003', 'WAP004', 'WAP005', 'WAP006', 'WAP007', 'WAP008', 'WAP009', 'WAP010', 'WAP011', 'WAP012', 'WAP013', 'WAP014', 'WAP015', 'WAP016', 'WAP017', 'WAP018', 'WAP019', 'WAP020'] ... total columnas=529

Conteo por clase en 'FLOOR':
FLOOR
3    5048
1    5002
2    4416
0    4369
4    1102
Name: count, dtype: int64

Número de clases distintas en 'FLOOR': 5


---

## Paso 2: Preparar los datos

**Instrucciones:**

- Elimina las columnas que no son relevantes para la tarea de clasificación del piso:
  - `LONGITUDE`, `LATITUDE`, `SPACEID`, `RELATIVEPOSITION`, `USERID`, `PHONEID`, `TIMESTAMP`
- Conserva únicamente:
  - Las columnas `WAP001` a `WAP520` como características (RSSI de puntos de acceso WiFi).
  - La columna `FLOOR` como variable objetivo.
- Verifica si existen valores atípicos o valores inválidos en las señales WiFi (por ejemplo: valores constantes como 100 o -110 que suelen indicar ausencia de señal).
- Separa el conjunto de datos en:
  - `X`: matriz de características (todas las columnas `WAP`)
  - `y`: vector objetivo (`FLOOR`)


In [2]:
# Paso 2: Preparar los datos (eliminar columnas irrelevantes, seleccionar WAPs y target)

# Columnas a eliminar según instrucción
cols_to_drop = ['LONGITUDE', 'LATITUDE', 'SPACEID', 'RELATIVEPOSITION', 'USERID', 'PHONEID', 'TIMESTAMP']

# Crear una copia limpia del dataframe original sin las columnas irrelevantes
df_clean = df.drop(columns=cols_to_drop, errors='ignore').copy()

# Determinar columnas WAP (WAP001..WAP520) existents en el dataframe
wap_cols = [f"WAP{str(i).zfill(3)}" for i in range(1, 521)]
wap_cols = [c for c in wap_cols if c in df_clean.columns]

# Verificar que la columna objetivo exista
if 'FLOOR' not in df_clean.columns:
    raise KeyError("La columna 'FLOOR' no se encontró en el dataset.")

# Construir X (características WAP) e y (target)
X = df_clean[wap_cols].copy()
y = df_clean['FLOOR'].copy()

# Información resumida de la preparación
print(f"Dimensiones originales: {df.shape}")
print(f"Dimensiones después de limpiar: X={X.shape}, y={y.shape}")
print(f"Número de columnas WAP detectadas: {len(wap_cols)}")

# Comprobaciones de valores atípicos / indicadores de ausencia de señal
count_100 = (X == 100).sum().sum()
count_minus_110 = (X == -110).sum().sum()
rssi_min = X.min().min()
rssi_max = X.max().max()

print(f"Total de entradas con valor 100 (ausencia): {count_100}")
print(f"Total de entradas con valor -110 (posible outlier): {count_minus_110}")
print(f"Rango de valores RSSI detectado en X: min={rssi_min}, max={rssi_max}")

# Cuenta de clases en y
print("\nDistribución de clases en 'FLOOR':")
print(y.value_counts().sort_index())

Dimensiones originales: (19937, 529)
Dimensiones después de limpiar: X=(19937, 520), y=(19937,)
Número de columnas WAP detectadas: 520
Total de entradas con valor 100 (ausencia): 10008477
Total de entradas con valor -110 (posible outlier): 0
Rango de valores RSSI detectado en X: min=-104, max=100

Distribución de clases en 'FLOOR':
FLOOR
0    4369
1    5002
2    4416
3    5048
4    1102
Name: count, dtype: int64


In [ ]:
# tu código aquí

--- 

## Paso 3: Preprocesamiento de las señales WiFi

**Contexto:**

Las columnas `WAP001` a `WAP520` representan la intensidad de la señal (RSSI) recibida desde distintos puntos de acceso WiFi. Los valores típicos de RSSI están en una escala negativa, donde:

- Valores cercanos a **0 dBm** indican señal fuerte.
- Valores cercanos a **-100 dBm** indican señal débil o casi ausente.
- Un valor de **100** en este dataset representa una señal **no detectada**, es decir, el punto de acceso no fue visto por el dispositivo en ese instante.

**Instrucciones:**

- Para facilitar el procesamiento y tratar la ausencia de señal de forma coherente, se recomienda mapear todos los valores **100** a **-100**, que semánticamente representa *ausencia de señal detectable*.
- Esto unifica el rango de valores y evita que 100 (un valor artificial) afecte negativamente la escala de los algoritmos.

**Pasos sugeridos:**

- Reemplaza todos los valores `100` por `-100` en las columnas `WAP001` a `WAP520`:
  ```python
  X[X == 100] = -100


In [3]:
# Paso 3: Preprocesamiento de las señales WiFi
# Reemplazar los 100 (no detectado) por -100 (ausencia de señal)
print("Total de entradas con valor 100 antes:", (X == 100).sum().sum())

X.replace(100, -100, inplace=True)

print("Total de entradas con valor 100 después:", (X == 100).sum().sum())
print("Rango de valores RSSI en X: min =", X.min().min(), ", max =", X.max().max())

Total de entradas con valor 100 antes: 10008477
Total de entradas con valor 100 después: 0
Rango de valores RSSI en X: min = -104 , max = 0


--- 

## Paso 4: Entrenamiento y optimización de hiperparámetros

**Objetivo:**

Entrenar y comparar distintos clasificadores para predecir correctamente el piso (`FLOOR`) y encontrar los mejores hiperparámetros para cada uno mediante validación cruzada.

**Clasificadores a evaluar:**

- K-Nearest Neighbors (KNN)
- Gaussian Naive Bayes
- Regresión Logística
- Árboles de Decisión
- Support Vector Machines (SVM)
- Random Forest

**Procedimiento:**

1. Divide el dataset en conjunto de **entrenamiento** (80%) y **validación** (20%) usando `train_test_split` con `stratify=y`.
2. Para cada clasificador:
   - Define el espacio de búsqueda de hiperparámetros.
   - Usa **validación cruzada 5-fold** sobre el conjunto de entrenamiento para seleccionar los mejores hiperparámetros.
   - Emplea una estrategia de búsqueda adecuada:
     - **GridSearchCV**: búsqueda exhaustiva (ideal para espacios pequeños).
     - **RandomizedSearchCV**: búsqueda aleatoria (más eficiente con espacios amplios).
     - **Bayesian Optimization** (opcional): para búsquedas más inteligentes, usando librerías como `optuna` o `skopt`.
3. Guarda el mejor modelo encontrado para cada clasificador con su configuración óptima.



In [4]:
from sklearn.model_selection import train_test_split

# Paso 4 (inicio): dividir el dataset en entrenamiento (80%) y validación (20%) con estratificación por y

X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

print("Dimensiones:")
print("  X_train:", X_train.shape, "y_train:", y_train.shape)
print("  X_val:  ", X_val.shape,   "y_val:  ", y_val.shape)

print("\nDistribución de clases (proporción) en y_train:")
print(y_train.value_counts(normalize=True).sort_index())

print("\nDistribución de clases (proporción) en y_val:")
print(y_val.value_counts(normalize=True).sort_index())

Dimensiones:
  X_train: (15949, 520) y_train: (15949,)
  X_val:   (3988, 520) y_val:   (3988,)

Distribución de clases (proporción) en y_train:
FLOOR
0    0.219136
1    0.250862
2    0.221519
3    0.253182
4    0.055301
Name: proportion, dtype: float64

Distribución de clases (proporción) en y_val:
FLOOR
0    0.219157
1    0.251003
2    0.221414
3    0.253260
4    0.055165
Name: proportion, dtype: float64


In [5]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
import numpy as np

# Espacio de búsqueda para KNN (pequeño, usamos GridSearchCV)
param_grid = {
    'n_neighbors': [3, 5, 7, 9, 11],
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan']
}

knn = KNeighborsClassifier()

# GridSearchCV con validación cruzada 5-fold
grid_search = GridSearchCV(
    knn,
    param_grid,
    cv=5,
    scoring='accuracy',
    n_jobs=-1,
    verbose=2
)

grid_search.fit(X_train, y_train)

print("Mejores hiperparámetros KNN:", grid_search.best_params_)
print("Mejor score de validación cruzada:", grid_search.best_score_)

# Evaluación en el conjunto de validación hold-out
best_knn = grid_search.best_estimator_
val_score = best_knn.score(X_val, y_val)
print("Accuracy en conjunto de validación:", val_score)

Fitting 5 folds for each of 20 candidates, totalling 100 fits
Mejores hiperparámetros KNN: {'metric': 'euclidean', 'n_neighbors': 3, 'weights': 'distance'}
Mejor score de validación cruzada: 0.9962380282534692
Accuracy en conjunto de validación: 0.9952357071213641


In [6]:
from sklearn.naive_bayes import GaussianNB
import numpy as np

# Espacio de búsqueda para GaussianNB (var_smoothing en escala log)
param_grid_gnb = {'var_smoothing': np.logspace(-9, -1, 9)}

gnb = GaussianNB()

# GridSearchCV ya está disponible en el notebook; usarlo con cv=5
grid_search_gnb = GridSearchCV(
    gnb,
    param_grid_gnb,
    cv=5,
    scoring='accuracy',
    n_jobs=-1,
    verbose=2
)

grid_search_gnb.fit(X_train, y_train)

print("Mejores hiperparámetros GNB:", grid_search_gnb.best_params_)
print("Mejor score CV (accuracy):", grid_search_gnb.best_score_)

# Guardar el mejor estimador y evaluar en el hold-out (X_val, y_val)
best_gnb = grid_search_gnb.best_estimator_
val_score_gnb = best_gnb.score(X_val, y_val)
print("Accuracy en validación hold-out:", val_score_gnb)

Fitting 5 folds for each of 9 candidates, totalling 45 fits
Mejores hiperparámetros GNB: {'var_smoothing': np.float64(0.01)}
Mejor score CV (accuracy): 0.7114555815395989
Accuracy en validación hold-out: 0.7091273821464393


In [10]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
import time

# Paso 4 (Árbol de Decisión): Entrenamiento y optimización de hiperparámetros

# Espacio de búsqueda para Árbol de Decisión (pequeño-medio, usamos GridSearchCV)
param_grid_dt = {
    'max_depth': [5, 10, 15, 20, None],
    'criterion': ['gini', 'entropy'],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

dt = DecisionTreeClassifier(random_state=42)

# GridSearchCV con validación cruzada 5-fold
grid_search_dt = GridSearchCV(
    dt,
    param_grid_dt,
    cv=5,
    scoring='accuracy',
    n_jobs=-1,
    verbose=2
)

print("Entrenando Árbol de Decisión con GridSearchCV...")
start_time = time.time()
grid_search_dt.fit(X_train, y_train)
train_time_dt = time.time() - start_time

print(f"\nTiempo de entrenamiento: {train_time_dt:.2f} segundos")
print("Mejores hiperparámetros DT:", grid_search_dt.best_params_)
print("Mejor score de validación cruzada:", grid_search_dt.best_score_)

# Guardar el mejor estimador y evaluar en el hold-out (X_val, y_val)
best_dt = grid_search_dt.best_estimator_
val_score_dt = best_dt.score(X_val, y_val)
print("Accuracy en conjunto de validación:", val_score_dt)


Entrenando Árbol de Decisión con GridSearchCV...
Fitting 5 folds for each of 90 candidates, totalling 450 fits

Tiempo de entrenamiento: 24.02 segundos
Mejores hiperparámetros DT: {'criterion': 'gini', 'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 5}
Mejor score de validación cruzada: 0.9659542844672764
Accuracy en conjunto de validación: 0.9711634904714143


In [11]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import RandomizedSearchCV
import numpy as np
import time

# Espacio de búsqueda para DecisionTree (más amplio, usamos RandomizedSearchCV)
param_dist_dt = {
    'max_depth': [5, 10, 15, 20, None],
    'criterion': ['gini', 'entropy'],
    'min_samples_split': np.arange(2, 21),
    'min_samples_leaf': np.arange(1, 11),
    'splitter': ['best', 'random']
}

dt = DecisionTreeClassifier(random_state=42)

random_search_dt = RandomizedSearchCV(
    dt,
    param_distributions=param_dist_dt,
    n_iter=30,
    cv=5,
    scoring='accuracy',
    n_jobs=-1,
    verbose=2,
    random_state=42
)

print("Entrenando DecisionTree con RandomizedSearchCV...")
start_time = time.time()
random_search_dt.fit(X_train, y_train)
train_time_dt = time.time() - start_time

print(f"\nTiempo de entrenamiento: {train_time_dt:.2f} segundos")
print("Mejores hiperparámetros DT:", random_search_dt.best_params_)
print("Mejor score de validación cruzada:", random_search_dt.best_score_)

# Evaluación en el conjunto de validación hold-out
best_dt = random_search_dt.best_estimator_
val_score_dt = best_dt.score(X_val, y_val)
print("Accuracy en conjunto de validación:", val_score_dt)

Entrenando DecisionTree con RandomizedSearchCV...
Fitting 5 folds for each of 30 candidates, totalling 150 fits

Tiempo de entrenamiento: 7.30 segundos
Mejores hiperparámetros DT: {'splitter': 'random', 'min_samples_split': np.int64(7), 'min_samples_leaf': np.int64(3), 'max_depth': None, 'criterion': 'entropy'}
Mejor score de validación cruzada: 0.9564241107018543
Accuracy en conjunto de validación: 0.9583751253761283


In [13]:
from sklearn.svm import SVC
from sklearn.model_selection import RandomizedSearchCV

param_dist_svm = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf'],
    'gamma': ['scale', 0.01, 0.1]
}

svm = SVC(random_state=42)

# Usar una muestra pequeña para tuning
X_train_small = X_train.sample(n=2000, random_state=42)
y_train_small = y_train.loc[X_train_small.index]

random_search_svm = RandomizedSearchCV(
    svm,
    param_distributions=param_dist_svm,
    n_iter=5,
    cv=3,
    scoring='accuracy',
    n_jobs=-1,
    verbose=2,
    random_state=42
)

print("Entrenando SVM con RandomizedSearchCV (versión rápida)...")
random_search_svm.fit(X_train_small, y_train_small)

print("Mejores hiperparámetros SVM:", random_search_svm.best_params_)
print("Mejor score de validación cruzada:", random_search_svm.best_score_)

# Evaluación en el conjunto de validación hold-out
best_svm = random_search_svm.best_estimator_
val_score_svm = best_svm.score(X_val, y_val)
print("Accuracy en conjunto de validación:", val_score_svm)

Entrenando SVM con RandomizedSearchCV (versión rápida)...
Fitting 3 folds for each of 5 candidates, totalling 15 fits
Mejores hiperparámetros SVM: {'kernel': 'linear', 'gamma': 'scale', 'C': 0.1}
Mejor score de validación cruzada: 0.9749989869929901
Accuracy en conjunto de validación: 0.9819458375125376


In [17]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV

# Usar una muestra para tuning (más rápido). Ajusta n_samples según tu RAM/tiempo.
n_samples = 5000
X_tune = X_train.sample(n=min(n_samples, len(X_train)), random_state=42)
y_tune = y_train.loc[X_tune.index]

# Espacio de búsqueda razonable y no excesivo
param_dist_rf = {
    'n_estimators': [100, 200, 300, 400],
    'max_depth': [None, 20, 30, 40],
    'max_features': ['sqrt', 'log2', 0.5],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False],
    'criterion': ['gini', 'entropy']
}

rf = RandomForestClassifier(random_state=42, n_jobs=-1)

# RandomizedSearchCV (cv=3 para acelerar)
rs = RandomizedSearchCV(
    rf,
    param_distributions=param_dist_rf,
    n_iter=20,
    cv=3,
    scoring='accuracy',
    n_jobs=-1,
    verbose=2,
    random_state=42
)

print("Iniciando búsqueda de hiperparámetros (muestra)...")
rs.fit(X_tune, y_tune)

print("\nMejores hiperparámetros encontrados (muestra):", rs.best_params_)
print("Mejor score CV en la muestra:", rs.best_score_)

# Reentrenar el modelo con los mejores hiperparámetros sobre TODO el X_train
best_params = rs.best_params_
best_rf = RandomForestClassifier(**best_params, random_state=42, n_jobs=-1)

print("\nReentrenando Random Forest con mejor configuración en todo X_train...")
best_rf.fit(X_train, y_train)

# Evaluación en hold-out
val_score_rf = best_rf.score(X_val, y_val)
print(f"Accuracy en conjunto de validación: {val_score_rf:.4f}")

# Guardar el objeto de búsqueda si se necesita más tarde
random_search_rf = rs


Iniciando búsqueda de hiperparámetros (muestra)...
Fitting 3 folds for each of 20 candidates, totalling 60 fits

Mejores hiperparámetros encontrados (muestra): {'n_estimators': 300, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': 40, 'criterion': 'gini', 'bootstrap': False}
Mejor score CV en la muestra: 0.9897998791598223

Reentrenando Random Forest con mejor configuración en todo X_train...
Accuracy en conjunto de validación: 0.9962


In [19]:
import joblib
from pathlib import Path
import datetime

# Guardar los mejores modelos (KNN, GaussianNB, LogisticRegression, DecisionTree, SVM, RandomForest)

models_dir = Path("models")
models_dir.mkdir(exist_ok=True)

timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")

models_to_save = {
    'KNN': globals().get('best_knn'),
    'GaussianNB': globals().get('best_gnb'),
    'LogisticRegression': globals().get('best_lr'),
    'DecisionTree': globals().get('best_dt'),
    'SVM': globals().get('best_svm'),
    'RandomForest': globals().get('best_rf'),
    'best_models_dict': globals().get('best_models')
}

saved_files = {}
for name, model in models_to_save.items():
    if model is None:
        print(f"Advertencia: '{name}' no existe en el entorno y no se guardará.")
        continue
    fname = models_dir / f"{name}_{timestamp}.joblib"
    joblib.dump(model, fname)
    saved_files[name] = str(fname)
    print(f"Guardado: {name} -> {fname}")

# Mostrar resumen
print("\nResumen de archivos guardados:")
for k, v in saved_files.items():
    print(f" - {k}: {v}")

Guardado: KNN -> models\KNN_20251203_124947.joblib
Guardado: GaussianNB -> models\GaussianNB_20251203_124947.joblib
Guardado: LogisticRegression -> models\LogisticRegression_20251203_124947.joblib
Guardado: DecisionTree -> models\DecisionTree_20251203_124947.joblib
Guardado: SVM -> models\SVM_20251203_124947.joblib
Guardado: RandomForest -> models\RandomForest_20251203_124947.joblib
Guardado: best_models_dict -> models\best_models_dict_20251203_124947.joblib

Resumen de archivos guardados:
 - KNN: models\KNN_20251203_124947.joblib
 - GaussianNB: models\GaussianNB_20251203_124947.joblib
 - LogisticRegression: models\LogisticRegression_20251203_124947.joblib
 - DecisionTree: models\DecisionTree_20251203_124947.joblib
 - SVM: models\SVM_20251203_124947.joblib
 - RandomForest: models\RandomForest_20251203_124947.joblib
 - best_models_dict: models\best_models_dict_20251203_124947.joblib


---

## Paso 5: Crear una tabla resumen de los mejores modelos

**Instrucciones:**

Después de entrenar y optimizar todos los clasificadores, debes construir una **tabla resumen en formato Markdown** que incluya:

- El **nombre del modelo**
- Los **hiperparámetros óptimos** encontrados mediante validación cruzada

### Requisitos:

- La tabla debe estar escrita en formato **Markdown**.
- Cada fila debe corresponder a uno de los modelos evaluados.
- Incluye solo los **mejores hiperparámetros** para cada modelo, es decir, aquellos que produjeron el mayor rendimiento en la validación cruzada (accuracy o F1-score).
- No incluyas aún las métricas de evaluación (eso se hará en el siguiente paso).

### Ejemplo de formato:


| Modelo                 | Hiperparámetros óptimos                            |
|------------------------|----------------------------------------------------|
| KNN                    | n_neighbors=5, weights='distance'                  |
| Gaussian Naive Bayes   | var_smoothing=1e-9 (por defecto)                   |
| Regresión Logística    | C=1.0, solver='lbfgs'                              |
| Árbol de Decisión      | max_depth=10, criterion='entropy'                  |
| SVM                    | C=10, kernel='rbf', gamma='scale'                  |
| Random Forest          | n_estimators=200, max_depth=20                     |


# tu tabla de resultados aquí

In [23]:
from pathlib import Path
from IPython.display import Markdown, display

# Paso 5: Crear y mostrar la tabla resumen (Markdown) con los mejores hiperparámetros

# Construir la tabla a partir de summary_table si ya existe, sino desde los modelos guardados en memoria
if 'summary_table' in globals():
    md_table = summary_table
else:
    bm = globals().get('best_models', {})
    rows = []

    # KNN
    knn = bm.get('KNN') or globals().get('best_knn')
    if knn is not None:
        rows.append((
            "K-Nearest Neighbors (KNN)",
            f"n_neighbors={getattr(knn,'n_neighbors',None)}, weights='{getattr(knn,'weights',None)}', metric='{getattr(knn,'metric',None)}'"
        ))

    # GaussianNB
    gnb = bm.get('GaussianNB') or globals().get('best_gnb')
    if gnb is not None:
        rows.append(("Gaussian Naive Bayes", f"var_smoothing={getattr(gnb,'var_smoothing',None)}"))

    # Logistic Regression (pipeline)
    lr = bm.get('LogisticRegression') or globals().get('best_lr')
    if lr is not None:
        clf = None
        try:
            clf = lr.named_steps.get('clf')  # pipeline
        except Exception:
            clf = lr
        if clf is not None:
            rows.append((
                "Regresión Logística",
                f"clf__C={getattr(clf,'C',None)}, clf__penalty='{getattr(clf,'penalty',None)}', clf__solver='{getattr(clf,'solver',None)}'"
            ))

    # Decision Tree
    dt = bm.get('DecisionTree') or globals().get('best_dt') or globals().get('dt')
    if dt is not None:
        rows.append((
            "Árbol de Decisión",
            f"criterion='{getattr(dt,'criterion',None)}', min_samples_split={getattr(dt,'min_samples_split',None)}, "
            f"min_samples_leaf={getattr(dt,'min_samples_leaf',None)}, splitter='{getattr(dt,'splitter',None)}', max_depth={getattr(dt,'max_depth',None)}"
        ))

    # SVM
    svm = bm.get('SVM') or globals().get('best_svm')
    if svm is not None:
        rows.append((
            "Support Vector Machines (SVM)",
            f"C={getattr(svm,'C',None)}, kernel='{getattr(svm,'kernel',None)}', gamma='{getattr(svm,'gamma',None)}'"
        ))

    # Random Forest
    rf = bm.get('RandomForest') or globals().get('best_rf') or globals().get('best_rf_full')
    if rf is not None:
        rows.append((
            "Random Forest",
            f"n_estimators={getattr(rf,'n_estimators',None)}, max_depth={getattr(rf,'max_depth',None)}, "
            f"max_features='{getattr(rf,'max_features',None)}', criterion='{getattr(rf,'criterion',None)}', "
            f"bootstrap={getattr(rf,'bootstrap',None)}, min_samples_split={getattr(rf,'min_samples_split',None)}, min_samples_leaf={getattr(rf,'min_samples_leaf',None)}"
        ))

    # Montar la tabla Markdown
    md_table = "| Modelo | Hiperparámetros óptimos |\n|---|---|\n"
    for name, hp in rows:
        md_table += f"| {name} | {hp} |\n"

# Mostrar la tabla en el notebook (Markdown) si es posible, y guardarla en models/best_models_summary.md
try:
    display(Markdown(md_table))
except Exception:
    print(md_table)

models_dir = Path("models")
models_dir.mkdir(exist_ok=True)
summary_path = models_dir / "best_models_summary.md"
summary_path.write_text(md_table)

print(f"\nTabla guardada en: {summary_path}")

| Modelo | Hiperparámetros óptimos |
|---|---|
| K-Nearest Neighbors (KNN) | n_neighbors=3, weights='distance', metric='euclidean' |
| Gaussian Naive Bayes | var_smoothing=np.float64(0.01) |
| Regresión Logística | clf__C=np.float64(32.90344562312671), clf__penalty='l2', clf__solver='lbfgs' |
| Árbol de Decisión | criterion='entropy', min_samples_split=np.int64(7), min_samples_leaf=np.int64(3), splitter='random', max_depth=None |
| Support Vector Machines (SVM) | C=0.1, kernel='linear', gamma='scale' |
| Random Forest | n_estimators=300, max_depth=40, max_features='sqrt', criterion='gini', bootstrap=False, min_samples_split=2, min_samples_leaf=1 |



Tabla guardada en: models\best_models_summary.md


---

## Paso 6: Preparar los datos finales para evaluación

**Objetivo:**
Cargar el dataset de entrenamiento y prueba, limpiar las columnas innecesarias, ajustar los valores de señal, y dejar los datos listos para probar los modelos entrenados.

**Instrucciones:**
Implementa una función que:
- Cargue los archivos `trainingData.csv` y `validationData.csv`
- Elimine las columnas irrelevantes (`LONGITUDE`, `LATITUDE`, `SPACEID`, `RELATIVEPOSITION`, `USERID`, `PHONEID`, `TIMESTAMP`)
- Reemplace los valores `100` por `-100` en las columnas `WAP001` a `WAP520`
- Separe las características (`X`) y la variable objetivo (`FLOOR`)
- Devuelva los conjuntos `X_train`, `X_test`, `y_train`, `y_test`

In [25]:
from pathlib import Path
import pandas as pd

def load_prepare_ujiindoorloc(training_path=None, validation_path=None,
                              drop_cols=None, wap_prefix='WAP', n_waps=520):
    """
    Carga y prepara los datasets trainingData.csv y validationData.csv:
    - elimina columnas irrelevantes (drop_cols si se proporciona, sino usa cols_to_drop del entorno si existe)
    - reemplaza 100 por -100 en columnas WAP001..WAP{n_waps}
    - separa X (WAPs) e y (FLOOR)
    Devuelve: X_train, X_test, y_train, y_test (pandas DataFrame/Series)
    """

    # Obtener lista de columnas a eliminar (usar variable existente si está en el notebook)
    if drop_cols is None:
        drop_cols = globals().get('cols_to_drop',
                                 ['LONGITUDE', 'LATITUDE', 'SPACEID', 'RELATIVEPOSITION', 'USERID', 'PHONEID', 'TIMESTAMP'])

    # función auxiliar para localizar archivos
    def _find_file(fname, provided):
        if provided:
            p = Path(provided)
            if p.exists():
                return p
            raise FileNotFoundError(f"Archivo especificado no existe: {provided}")
        # buscar en candidate_paths si existe en el entorno
        cand = globals().get('candidate_paths', [])
        for p in cand:
            try:
                if Path(p).exists() and Path(p).name.lower() == fname.lower():
                    return Path(p)
            except Exception:
                continue
        # rutas comunes
        common = [Path(fname), Path('data')/fname, Path('datasets/UJIIndoorLoc')/fname, Path('./')/fname]
        for p in common:
            if p.exists():
                return p
        raise FileNotFoundError(f"No se encontró {fname} en rutas comunes. Pasa la ruta como parámetro.")

    train_fp = _find_file('trainingData.csv', training_path)
    val_fp = _find_file('validationData.csv', validation_path)

    df_train = pd.read_csv(train_fp)
    df_val = pd.read_csv(val_fp)

    # eliminar columnas irrelevantes (si están)
    df_train = df_train.drop(columns=drop_cols, errors='ignore').copy()
    df_val = df_val.drop(columns=drop_cols, errors='ignore').copy()

    # determinar columnas WAP presentes
    wap_cols = [f"{wap_prefix}{str(i).zfill(3)}" for i in range(1, n_waps + 1)]
    wap_cols = [c for c in wap_cols if c in df_train.columns and c in df_val.columns]
    if not wap_cols:
        raise KeyError("No se detectaron columnas WAP en los archivos proporcionados.")

    # reemplazar 100 -> -100 (ausencia de señal)
    df_train.loc[:, wap_cols] = df_train.loc[:, wap_cols].replace(100, -100)
    df_val.loc[:, wap_cols] = df_val.loc[:, wap_cols].replace(100, -100)

    # verificar target
    if 'FLOOR' not in df_train.columns or 'FLOOR' not in df_val.columns:
        raise KeyError("La columna 'FLOOR' no se encontró en uno de los datasets.")

    X_train = df_train[wap_cols].copy()
    y_train = df_train['FLOOR'].copy()
    X_test = df_val[wap_cols].copy()
    y_test = df_val['FLOOR'].copy()

    # breve resumen
    print(f"Archivos cargados: {train_fp.name} ({df_train.shape}), {val_fp.name} ({df_val.shape})")
    print(f"WAPs usadas: {len(wap_cols)} (primeras 5: {wap_cols[:5]})")
    print(f"Rango RSSI train: {X_train.min().min()} .. {X_train.max().max()}")
    print(f"Rango RSSI test:  {X_test.min().min()} .. {X_test.max().max()}")

    return X_train, X_test, y_train, y_test

---

## Paso 7: Evaluar modelos optimizados en el conjunto de prueba

**Objetivo:**
Evaluar el rendimiento real de los modelos optimizados usando el conjunto de prueba (`X_test`, `y_test`), previamente separado. Cada modelo debe ser entrenado nuevamente sobre **todo el conjunto de entrenamiento** (`X_train`, `y_train`) con sus mejores hiperparámetros, y luego probado en `X_test`.

**Instrucciones:**

1. Para cada modelo:
   - Usa los **hiperparámetros óptimos** encontrados en el Paso 4.
   - Entrena el modelo con `X_train` y `y_train`.
   - Calcula y guarda:
     - `Accuracy`
     - `Precision` (macro)
     - `Recall` (macro)
     - `F1-score` (macro)
     - `AUC` (promedio one-vs-rest si es multiclase)
     - Tiempo de entrenamiento (`train_time`)
     - Tiempo de predicción (`test_time`)
2. Muestra todos los resultados en una **tabla comparativa**


In [30]:
from sklearn.base import clone
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.preprocessing import label_binarize
from sklearn.calibration import CalibratedClassifierCV
from pathlib import Path
import time
import numpy as np
import pandas as pd
import joblib

# Paso 7: Evaluar modelos optimizados en el conjunto de prueba

# Cargar y preparar los datos finales (training + validation files)
X_train_full, X_test, y_train_full, y_test = load_prepare_ujiindoorloc()

# Obtener los mejores modelos/hyperparámetros encontrados
models_to_eval = globals().get('best_models') or globals().get('models_dict') or {}

results = []
models_dir = Path("models")
models_dir.mkdir(exist_ok=True)

classes = np.unique(y_test)

for name, model_template in models_to_eval.items():
    print(f"Evaluando: {name}")
    estimator = clone(model_template)

    # Entrenar en todo el conjunto de entrenamiento
    t0 = time.time()
    estimator.fit(X_train_full, y_train_full)
    train_time = time.time() - t0

    # Predicción en test (medir tiempo)
    t0 = time.time()
    y_pred = estimator.predict(X_test)
    test_time = time.time() - t0

    # Métricas básicas
    acc = accuracy_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred, average='macro', zero_division=0)
    rec = recall_score(y_test, y_pred, average='macro', zero_division=0)
    f1 = f1_score(y_test, y_pred, average='macro', zero_division=0)

    # AUC (manejar predict_proba/decision_function; intentar calibrar si no disponible)
    auc = np.nan
    try:
        y_score = None
        if hasattr(estimator, "predict_proba"):
            y_score = estimator.predict_proba(X_test)
        elif hasattr(estimator, "decision_function"):
            y_score = estimator.decision_function(X_test)
        else:
            # intentar calibrar para obtener probabilidades
            try:
                calib = CalibratedClassifierCV(base_estimator=clone(model_template), cv=3)
                calib.fit(X_train_full, y_train_full)
                y_score = calib.predict_proba(X_test)
            except Exception:
                y_score = None

        if y_score is not None:
            # Caso binario: y_score 1D o (n_samples,1)
            if y_score.ndim == 1 or (y_score.ndim == 2 and y_score.shape[1] == 1):
                # asegurar array 1D
                y_sc = y_score.ravel()
                auc = roc_auc_score(y_test, y_sc)
            else:
                # multiclass: binarizar y_test y calcular OVR macro
                y_test_bin = label_binarize(y_test, classes=classes)
                # si label_binarize devuelve una sola columna para binario, tratarlo
                if y_test_bin.ndim == 1 or (y_test_bin.ndim == 2 and y_test_bin.shape[1] == 1):
                    auc = roc_auc_score(y_test, y_score.ravel())
                else:
                    auc = roc_auc_score(y_test_bin, y_score, multi_class='ovr', average='macro')
    except Exception:
        auc = np.nan

    # Guardar modelo entrenado
    saved_path = models_dir / f"{name}_final.joblib"
    joblib.dump(estimator, saved_path)

    results.append({
        'model': name,
        'train_time_s': train_time,
        'test_time_s': test_time,
        'accuracy': acc,
        'precision_macro': prec,
        'recall_macro': rec,
        'f1_macro': f1,
        'auc_ovr_macro': auc,
        'saved_model_path': str(saved_path)
    })

# Construir dataframe de resultados, ordenar por accuracy descendente y guardarlo
results_df = pd.DataFrame(results).sort_values(by='accuracy', ascending=False).reset_index(drop=True)
results_csv = models_dir / "evaluation_results.csv"
results_md = models_dir / "evaluation_results.md"

results_df.to_csv(results_csv, index=False)

# Crear tabla Markdown simple
md = "| modelo | train_time_s | test_time_s | accuracy | precision_macro | recall_macro | f1_macro | auc_ovr_macro | saved_model_path |\n"
md += "|---|---:|---:|---:|---:|---:|---:|---:|---|\n"
for _, r in results_df.iterrows():
    auc_val = f"{r['auc_ovr_macro']:.6f}" if not pd.isna(r['auc_ovr_macro']) else 'NaN'
    md += f"| {r['model']} | {r['train_time_s']:.3f} | {r['test_time_s']:.3f} | {r['accuracy']:.4f} | {r['precision_macro']:.4f} | {r['recall_macro']:.4f} | {r['f1_macro']:.4f} | {auc_val} | {r['saved_model_path']} |\n"
results_md.write_text(md)

# Mostrar resumen
print("\nResultados de evaluación (ordenados por accuracy):")
display(results_df)

print(f"\nCSV guardado en: {results_csv}")
print(f"Markdown guardado en: {results_md}")

Archivos cargados: trainingData.csv ((19937, 522)), validationData.csv ((1111, 522))
WAPs usadas: 520 (primeras 5: ['WAP001', 'WAP002', 'WAP003', 'WAP004', 'WAP005'])
Rango RSSI train: -104 .. 0
Rango RSSI test:  -102 .. -34
Evaluando: KNN
Evaluando: GaussianNB
Evaluando: LogisticRegression


c:\Users\kenet\Documents\keneth_bracamonte_clase_IA\.venv\Lib\site-packages\sklearn\linear_model\_logistic.py:1272: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.8. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


Evaluando: DecisionTree
Evaluando: SVM
Evaluando: RandomForest

Resultados de evaluación (ordenados por accuracy):


,model,train_time_s,test_time_s,accuracy,precision_macro,recall_macro,f1_macro,auc_ovr_macro,saved_model_path
0,RandomForest,1.775390,0.048880,0.911791,0.925350,0.885617,0.900750,0.987618,models\RandomForest_final.joblib
1,KNN,0.035920,0.364792,0.907291,0.920505,0.900514,0.907909,0.946595,models\KNN_final.joblib
2,LogisticRegression,2.478427,0.004609,0.891989,0.873486,0.897204,0.882796,0.970602,models\LogisticRegression_final.joblib
3,SVM,2.799414,0.064311,0.883888,0.873658,0.890209,0.880999,0.956739,models\SVM_final.joblib
4,DecisionTree,0.336492,0.002774,0.779478,0.773922,0.792129,0.776083,0.887806,models\DecisionTree_final.joblib
5,GaussianNB,0.151523,0.028804,0.775878,0.696839,0.799904,0.716969,0.936128,models\GaussianNB_final.joblib



CSV guardado en: models\evaluation_results.csv
Markdown guardado en: models\evaluation_results.md


In [ ]:
# tu código aquí

---
## Paso 8: Selección y justificación del mejor modelo

**Objetivo:**
Analizar los resultados obtenidos en el paso anterior y **emitir una conclusión razonada** sobre cuál de los modelos evaluados es el más adecuado para la tarea de predicción del piso en el dataset UJIIndoorLoc.

**Instrucciones:**

- Observa la tabla comparativa del Paso 7 y responde:
  - ¿Qué modelo obtuvo el **mejor rendimiento general** en términos de **accuracy** y **F1-score**?
  - ¿Qué tan consistente fue su rendimiento en **precision** y **recall**?
  - ¿Tiene un **tiempo de entrenamiento o inferencia** excesivamente alto?
  - ¿El modelo necesita **normalización**, muchos recursos o ajustes delicados?
- Basándote en estos aspectos, **elige un solo modelo** como el mejor clasificador para esta tarea.
- **Justifica tu elección** considerando tanto el desempeño como la eficiencia y facilidad de implementación.


# tu respuesta aquí

El mejor modelo es Random Forest. 

Obtuvo el mejor rendimiento en términos de accuracy (0.9118) y F1-score (0.9008). 

Su precisión y recall fueron consistentes, con valores de 0.9253 y 0.8856, respectivamente. 

Aunque su tiempo de entrenamiento (1.775s) y predicción (0.049s) es mayor que otros modelos como KNN, sigue siendo razonable. 

Random Forest ofrece un buen equilibrio entre precisión y robustez, aunque consume más memoria y es menos interpretable.

---

## Rúbrica de Evaluación

| Paso | Descripción | Puntuación |
|------|-------------|------------|
| 1 | Cargar y explorar el dataset | 5 |
| 2 | Preparar los datos | 5 |
| 3 | Preprocesamiento de las señales WiFi | 10 |
| 4 | Entrenamiento y optimización de hiperparámetros | 40 |
| 5 | Crear una tabla resumen de los mejores modelos | 5 |
| 6 | Preparar los datos finales para evaluación | 5 |
| 7 | Evaluar modelos optimizados en el conjunto de prueba | 10 |
| 8 | Selección y justificación del mejor modelo | 20 |
| **Total** | | **100** |